In [1]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import requests
import re
import os
import pandas as pd
from time import gmtime, strftime
from models.RSS_classifier import *

In [2]:
def get_articles_and_xml(url):
    page = requests.get(url)

    pattern = '<item rdf:about=.*?</item>'
    article_list = re.findall(pattern, page.text, flags=(re.MULTILINE | re.DOTALL))
    return article_list, page.text

In [3]:
def extract_item_info(xml_string):
    item_dict = {}
    item_dict['url'] = re.findall('http://arxiv.org/abs/\d{1,8}\.\d{1,8}', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['title'] = (re.findall('<title>(.*?)[(]arXiv:', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]).strip()
    item_dict['abstract'] = re.findall('<description rdf:.*?>(.*?)</description>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['authors'] = re.findall('<dc:creator>(.*?)</dc:creator>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    item_dict['authors'] = re.findall('\&quot;\&gt;(.*?)\&lt;/a\&gt;', item_dict['authors'], flags=(re.MULTILINE | re.DOTALL))
    item_dict['authors'] = ', '.join(item_dict['authors'])
    item_dict['full_text'] = xml_string
    return item_dict

In [4]:
def get_feed_parts(xml_string):
    beginning = re.findall('.*?<rdf:Seq>\n', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    middle = re.findall('</rdf:Seq>.*?</image>', xml_string, flags=(re.MULTILINE | re.DOTALL))[0]
    end = '</rdf:RDF>'
    
    beginning_good = re.sub('<title>(.*?)</title>', '<title>Good papers from arXiv</title>', beginning, flags=(re.MULTILINE | re.DOTALL))
    beginning_bad = re.sub('<title>(.*?)</title>', '<title>Boring papers from arXiv</title>', beginning, flags=(re.MULTILINE | re.DOTALL))
    
    beginning_good = re.sub('<description(.*?)>(.*?)</description>', '<description\g<1>>The Good Papers on ML, AI and Statistics</description>', beginning_good, flags=(re.MULTILINE | re.DOTALL))
    beginning_bad = re.sub('<description(.*?)>(.*?)</description>', '<description\g<1>>The Bad Papers on ML, AI and Statistics</description>', beginning_bad, flags=(re.MULTILINE | re.DOTALL))
    
    return beginning_good, beginning_bad, middle, end

In [5]:
def build_feeds(xml_string, articles_info_classified_pd):
    beginning_good, beginning_bad, middle, end = get_feed_parts(xml_string)
    
    
    items_good = []
    items_bad = []
    for i in range(len(articles_info_classified_pd)):
        item = '<rdf:li rdf:resource="' + articles_info_classified_pd['url'].iloc[i] + '"/>'
        if articles_info_classified_pd['class'].iloc[i] == True:
            items_good.append(item)
        else:
            items_bad.append(item)
            
    items_good = '\n'.join(items_good)
    items_bad = '\n'.join(items_bad)
    
    abstracts_good = []
    abstracts_bad = []
    for i in range(len(articles_info_classified_pd)):
        item = articles_info_classified_pd['full_text'].iloc[i]
        if articles_info_classified_pd['class'].iloc[i] == True:
            abstracts_good.append(item)
        else:
            abstracts_bad.append(item)
            
    abstracts_good = '\n'.join(abstracts_good)
    abstracts_bad = '\n'.join(abstracts_bad)
    
    good_text = beginning_good + items_good + middle + abstracts_good + end
    bad_text = beginning_bad + items_bad + middle + abstracts_bad + end
    
    return good_text, bad_text

In [10]:
urls = [
    'http://arxiv.org/rss/cs.NE',
    'http://arxiv.org/rss/cs.AI',
    'http://arxiv.org/rss/stat.ML'
]
good_feed_name = 'papers_good.xml'
bad_feed_name = 'papers_bad.xml'

articles_info_classified_list = []
for url in urls:
    article_list, xml_string = get_articles_and_xml(url)

    articles_info_list = []
    for article in article_list:
        info = extract_item_info(article)
        articles_info_list.append(info)

    articles_info_pd = pd.DataFrame(articles_info_list)
    articles_info_classified_list.append(classify(articles_info_pd))
    #articles_info_classified_pd = classify(articles_info_pd)

articles_info_classified_pd = pd.concat([articles_info_classified_list[0], 
                                              articles_info_classified_list[1],
                                              articles_info_classified_list[2]])

articles_info_classified_pd = articles_info_classified_pd.drop_duplicates(subset=['url'])

{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])}


/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])}


/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


{'svc': Pipeline(steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])}


/home/piotr.bednarski/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [11]:
articles_info_classified_pd

,abstract,authors,full_text,title,url,class
0,&lt;p&gt;After providing a brief historical ov...,"Dario Izzo, Christopher Sprague, Dharmesh Tailor","<item rdf:about=""http://arxiv.org/abs/1802.001...",Machine learning and evolutionary techniques i...,http://arxiv.org/abs/1802.00180,False
1,&lt;p&gt;We propose an architecture for VQA wh...,"Ahmed Osman, Wojciech Samek","<item rdf:about=""http://arxiv.org/abs/1802.002...",Dual Recurrent Attention Units for Visual Ques...,http://arxiv.org/abs/1802.00209,True
2,&lt;p&gt;It will be shown that according to th...,Dietmar Volz,"<item rdf:about=""http://arxiv.org/abs/1308.160...",A Note on Topology Preservation in Classificat...,http://arxiv.org/abs/1308.1603,True
0,&lt;p&gt;Negative affect is a proxy for mental...,"Mawulolo K. Ameko, Lihua Cai, Mehdi Boukhechba...","<item rdf:about=""http://arxiv.org/abs/1802.000...",Cluster-based Approach to Improve Affect Recog...,http://arxiv.org/abs/1802.00029,False
1,&lt;p&gt;We describe the first automatic appro...,Peter Sch&amp;#xfc;ller,"<item rdf:about=""http://arxiv.org/abs/1802.000...",Technical Report: Adjudication of Coreference ...,http://arxiv.org/abs/1802.00033,False
2,&lt;p&gt;Deceptive games are games where the r...,"Damien Anderson, Matthew Stephenson, Julian To...","<item rdf:about=""http://arxiv.org/abs/1802.000...",Deceptive Games.,http://arxiv.org/abs/1802.00048,True
3,&lt;p&gt;When humans perform inductive learnin...,"Lior Friedman, Shaul Markovitch","<item rdf:about=""http://arxiv.org/abs/1802.000...",Recursive Feature Generation for Knowledge-bas...,http://arxiv.org/abs/1802.00050,True
5,&lt;p&gt;The study and publication of historic...,"Sahar Aljalbout, Gilles Falquet","<item rdf:about=""http://arxiv.org/abs/1802.002...",pv:SSCategory.,http://arxiv.org/abs/1802.00295,False
6,&lt;p&gt;Tactical driving decision making is c...,"Jingchu Liu, Pengfei Hou, Lisen Mu, Yinan Yu, ...","<item rdf:about=""http://arxiv.org/abs/1802.003...",Elements of Effective Deep Reinforcement Learn...,http://arxiv.org/abs/1802.00332,False
7,&lt;p&gt;We investigate the automatic classifi...,Amitabha Karmakar,"<item rdf:about=""http://arxiv.org/abs/1802.003...",Classifying medical notes into standard diseas...,http://arxiv.org/abs/1802.00382,False


In [12]:
print("Gooood len: " +str(len(articles_info_classified_pd[articles_info_classified_pd['class']==True])))
print("Boring len: " +str(len(articles_info_classified_pd[articles_info_classified_pd['class']==False])))

Gooood len: 19
Boring len: 31


,abstract,authors,full_text,title,url,class
0,&lt;p&gt;After providing a brief historical ov...,"Dario Izzo, Christopher Sprague, Dharmesh Tailor","<item rdf:about=""http://arxiv.org/abs/1802.001...",Machine learning and evolutionary techniques i...,http://arxiv.org/abs/1802.00180,True
1,&lt;p&gt;We propose an architecture for VQA wh...,"Ahmed Osman, Wojciech Samek","<item rdf:about=""http://arxiv.org/abs/1802.002...",Dual Recurrent Attention Units for Visual Ques...,http://arxiv.org/abs/1802.00209,True
2,&lt;p&gt;It will be shown that according to th...,Dietmar Volz,"<item rdf:about=""http://arxiv.org/abs/1308.160...",A Note on Topology Preservation in Classificat...,http://arxiv.org/abs/1308.1603,False
0,&lt;p&gt;Negative affect is a proxy for mental...,"Mawulolo K. Ameko, Lihua Cai, Mehdi Boukhechba...","<item rdf:about=""http://arxiv.org/abs/1802.000...",Cluster-based Approach to Improve Affect Recog...,http://arxiv.org/abs/1802.00029,False
1,&lt;p&gt;We describe the first automatic appro...,Peter Sch&amp;#xfc;ller,"<item rdf:about=""http://arxiv.org/abs/1802.000...",Technical Report: Adjudication of Coreference ...,http://arxiv.org/abs/1802.00033,False
2,&lt;p&gt;Deceptive games are games where the r...,"Damien Anderson, Matthew Stephenson, Julian To...","<item rdf:about=""http://arxiv.org/abs/1802.000...",Deceptive Games.,http://arxiv.org/abs/1802.00048,True
3,&lt;p&gt;When humans perform inductive learnin...,"Lior Friedman, Shaul Markovitch","<item rdf:about=""http://arxiv.org/abs/1802.000...",Recursive Feature Generation for Knowledge-bas...,http://arxiv.org/abs/1802.00050,True
5,&lt;p&gt;The study and publication of historic...,"Sahar Aljalbout, Gilles Falquet","<item rdf:about=""http://arxiv.org/abs/1802.002...",pv:SSCategory.,http://arxiv.org/abs/1802.00295,False
6,&lt;p&gt;Tactical driving decision making is c...,"Jingchu Liu, Pengfei Hou, Lisen Mu, Yinan Yu, ...","<item rdf:about=""http://arxiv.org/abs/1802.003...",Elements of Effective Deep Reinforcement Learn...,http://arxiv.org/abs/1802.00332,False
7,&lt;p&gt;We investigate the automatic classifi...,Amitabha Karmakar,"<item rdf:about=""http://arxiv.org/abs/1802.003...",Classifying medical notes into standard diseas...,http://arxiv.org/abs/1802.00382,True


In [ ]:
good_text, bad_text = build_feeds(xml_string, articles_info_classified_pd)

with open(good_feed_name, 'w') as good:
    good.write(good_text)
    
with open(bad_feed_name, 'w') as bad:
    bad.write(bad_text)

In [ ]:
import git
repo = git.Repo( '../rss-classifier' )
print(repo.git.add( '.' ))

timestring = strftime("%Y%m%d_%H%M%S", gmtime())
message = 'Update at ' + timestring

print(repo.git.commit( m=message ))
print(repo.git.push())
print(repo.git.status())

In [17]:
articles_info_classified_pd[['title', 'class']]

,title,class
0,Machine learning and evolutionary techniques i...,False
1,Dual Recurrent Attention Units for Visual Ques...,True
2,A Note on Topology Preservation in Classificat...,True
0,Cluster-based Approach to Improve Affect Recog...,False
1,Technical Report: Adjudication of Coreference ...,False
2,Deceptive Games.,True
3,Recursive Feature Generation for Knowledge-bas...,True
5,pv:SSCategory.,False
6,Elements of Effective Deep Reinforcement Learn...,False
7,Classifying medical notes into standard diseas...,False


In [18]:
articles_info_classified_pd_new['class_old'] = articles_info_classified_pd['class']

In [24]:
articles_info_classified_pd_new[['title', 'class', 'class_old']]

,title,class,class_old
0,Machine learning and evolutionary techniques i...,True,False
1,Dual Recurrent Attention Units for Visual Ques...,True,True
2,A Note on Topology Preservation in Classificat...,False,True
0,Cluster-based Approach to Improve Affect Recog...,False,False
1,Technical Report: Adjudication of Coreference ...,False,False
2,Deceptive Games.,True,True
3,Recursive Feature Generation for Knowledge-bas...,True,True
5,pv:SSCategory.,False,False
6,Elements of Effective Deep Reinforcement Learn...,False,False
7,Classifying medical notes into standard diseas...,True,False
